# Goal

The goal of this notebook is to look at the power usage for rectangular matrices.

In [1]:
# Data preprocessing

import os

FREQUENCY = 905250000 # Available Frequencies: 905250000 and 1377000000
DEVICE = "AGX"

path = "./data/"+DEVICE+"/non-square/"+str(FREQUENCY)+"/"

files = os.listdir(path)

data = []

for file_name in files:
  temp = {
    # Inputs
    "device": "",
    "datatype": "",
    "matrix_rows": -1,
    "matrix_cols": -1,
    "tensor": None,
    "gpu_frequency": -1,

    # Results
    "power_usage": [],
    "flops": -1,

    # Calculates Results
    "avg_power": -1,
    "flops_per_watt": -1
  }
  with open(path+file_name) as f:
    temp['device'], temp['datatype'], temp['matrix_rows'], temp['matrix_cols'], temp['tensor'], temp['gpu_frequency'] = file_name.split('.')[0].split('-')
    temp['matrix_rows'] = int(temp['matrix_rows'])
    temp['matrix_cols'] = int(temp['matrix_cols'])
    temp['tensor'] = True if temp['tensor'] == "tensor" else False
    temp['gpu_frequency'] = int(temp['gpu_frequency'])

    file_data = f.readlines()

    _, temp['power_usage'] = zip(*[d.strip().split(",") for d in file_data[1:-1]])
    temp['power_usage'] = list(map(float, temp['power_usage']))
    temp['avg_power'] = sum(temp['power_usage'])/len(temp['power_usage'])

    temp['flops'] = float(file_data[-1])

    temp['flops_per_watt'] = temp['flops'] / temp['avg_power']
  data.append(temp)
  
print(len(data), "files processed.")

4096 files processed.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget
# import ipywidgets as widgets
# from IPython.display import display
# plt.ion()

search = {
    "device": DEVICE,
    "datatype": "float",
    "matrix_rows": -1, # 64-1024, step=64
    # "matrix_cols": -1, # 8-2048, step=8
    "tensor": True,
    "gpu_frequency": FREQUENCY,
}

fig, ax = plt.subplots(1, 1, figsize=(10, 7))
ax.set_axisbelow(True)
ax.grid(axis='y')
ax.set_xticks(range(0, 2048+1, 128))
ax.set_title("Flops per Watt by Matrix Size")
ax.set_xlabel("Columns")
ax.set_ylabel("Flops per Watt")
# ax.set_prop_cycle('color', [plt.get_cmap('gist_rainbow')(1.*i/16) for i in range(16)])

for row in range(64, 1024+1, 64):
  search['matrix_rows'] = row
  results = [d for d in data if search.items() <= d.items() ]
  x, y = zip(*sorted([(r['matrix_cols'], r['flops_per_watt']) for r in results], key=lambda d : d[0]))
  ax.plot(x, y, label=str(row)+" rows", linestyle='-' if row > 682 else '--' if row > 341 else '-.')

ax.legend(loc='upper left')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …